In [1]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama

from datasets import load_dataset

import cassio

/Users/saaijeeshsn/Desktop/Langchain/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/saaijeeshsn/Desktop/Langchain/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from PyPDF2 import PdfReader

In [4]:
ASTRA_DB_APPLICATION_TOKEN="AstraCS:LwvJNsFwFeCFFfygMAouoodn:07f3e1bf483864e97827f8e58eb58ac57dd35c769ea09b905322fa875a42b517"
ASTRA_DB_ID="3caff646-89f5-4bba-a9c3-6ecae898835f"


In [6]:
pdfreader=PdfReader('/Users/saaijeeshsn/Desktop/Langchain/pdf_query_langchain/data_warehousing.pdf')

In [7]:
from typing_extensions import Concatenate

raw_text=''
for i,page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text+=content

In [9]:
raw_text

'Data W arehousing\nRajan Tiwari,\nPublished by - Jharkhand Rai University \nSubject: DATA WAREHOUSING  Credits: 4\nSYLLABUS\nBasic Concepts of Data Warehousing\nIntroduction, Meaning and characteristics of Data Wa rehousing, Online Transaction Processing (OLTP), Data \nWarehousing Models, Data warehouse architectur e & Principles of Data Warehousing Data Mining. \n \nBuilding a Data Warehouse Project\nStructure of the Data warehouse, Data warehousing and Operational Systems,  Organizing for building data \nwarehousing, Important considerations – Tighter integration, Empowerment, Willingness Business \nConsiderations: Return on Investment Design Conside rations, Technical Conside ration, Implementation \nConsideration, Benefits of Data warehousing.  \nManaging and Implementing a Data Warehouse Project\nProject Management Process,  Scope Statement, Work Breakdown Structure  and Integration, Initiating a data \nwarehousing project Project Estimation,  Analyzing Probability and Risk, Mana

In [10]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN,database_id=ASTRA_DB_ID)

In [11]:
llm=Ollama(model="llama2")
embeddings=OllamaEmbeddings(model="mistral")


In [13]:
astra_vector_store= Cassandra(
    embedding=embeddings,
    table_name="qa_mini",
    session=None,
    keyspace=None,
)

In [14]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

Created a chunk of size 1008, which is longer than the specified 800
Created a chunk of size 973, which is longer than the specified 800
Created a chunk of size 1169, which is longer than the specified 800
Created a chunk of size 1093, which is longer than the specified 800
Created a chunk of size 975, which is longer than the specified 800
Created a chunk of size 809, which is longer than the specified 800
Created a chunk of size 1022, which is longer than the specified 800
Created a chunk of size 923, which is longer than the specified 800
Created a chunk of size 1073, which is longer than the specified 800
Created a chunk of size 844, which is longer than the specified 800
Created a chunk of size 1260, which is longer than the specified 800
Created a chunk of size 884, which is longer than the specified 800
Created a chunk of size 1445, which is longer than the specified 800
Created a chunk of size 931, which is longer than the specified 800
Created a chunk of size 1346, which is lo

In [15]:
texts[:50]

['Data W arehousing\nRajan Tiwari,\nPublished by - Jharkhand Rai University \nSubject: DATA WAREHOUSING  Credits: 4\nSYLLABUS\nBasic Concepts of Data Warehousing\nIntroduction, Meaning and characteristics of Data Wa rehousing, Online Transaction Processing (OLTP), Data \nWarehousing Models, Data warehouse architectur e & Principles of Data Warehousing Data Mining. \n \nBuilding a Data Warehouse Project\nStructure of the Data warehouse, Data warehousing and Operational Systems,  Organizing for building data \nwarehousing, Important considerations – Tighter integration, Empowerment, Willingness Business \nConsiderations: Return on Investment Design Conside rations, Technical Conside ration, Implementation \nConsideration, Benefits of Data warehousing.  \nManaging and Implementing a Data Warehouse Project',
 'Considerations: Return on Investment Design Conside rations, Technical Conside ration, Implementation \nConsideration, Benefits of Data warehousing.  \nManaging and Implementing a Da

In [17]:
astra_vector_store.add_texts(texts[:50])
print("Inserted %i headlines." % len(texts[:50]))

astra_vector_index= VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 50 headlines.


In [18]:
first_question = True
while True:
    if first_question:
        query_text = input("\n Enter your question (or type 'quit to exit):").strip()
    else:
       query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower()=="quit":
        break

    if query_text =="":
        continue

    first_question = False
    print("\nQuestion: \"%s\""%query_text)
    answer = astra_vector_index.query(query_text,llm=llm).strip()
    print("Answer: \"%s\"\n"%answer) 

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc,score in astra_vector_store.similarity_search_with_score(query_text,k=4):
        print("  [%0.4f] \"%s ...\"" %(score,doc.page_content[:84]))



Question: "what is the difference between olap and oltp"
Answer: "OLTP (Online Transaction Processing) and OLAP (Online Analytical Processing) are two different database systems that serve distinct purposes. Here's a summary of their main differences:

OLTP:

1. Focuses on handling large volumes of transactional data, such as financial transactions, customer orders, or inventory management.
2. Designed for fast and efficient data entry, update, and retrieval of small to medium-sized datasets.
3. Optimized for quick querying and processing of simple queries, such as "what is the current balance of a customer's account?"
4. Typically uses relational database management systems (RDBMS) like MySQL, Oracle, or PostgreSQL.
5. Data is stored in a flat file structure, with little to no nesting or hierarchical relationships between data elements.

OLAP:

1. Designed for complex analytical tasks, such as data aggregation, filtering, and reporting.
2. Optimized for handling large datasets and co